In [2]:
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.dates as mdates

def makeNullRects(dates, y):
    '''This function returns a list of matplotlib.patches.Rectangles where
    np.nan values are present in the y array. If values are consecutive,
    the rectangles will widen as needed.
    Note that this function is made for a figure with an x-axis of dates
    Input:
        dates: x axis date time values
        y: y axis range values as np.array, contains np.nan values

    Returns:
        list of matplotlib.patches.Rectangles located where
        y has np.nan values.

    Rectangle Parameters in function:
        opacityCoeff: how solid rectangles appear
        longRectColor: the color of the rectangles with >=7 width
        shortRectColor: the color of the rectanges with <7 width
    '''
    # setting up rectangle parameters
    opacityCoeff = 0.5
    longRectColor = "red"
    shortRectColor = "magenta"

    # prep work for creating rectangles for nan values
    index = 0
    yMax = np.nanmax(y)
    yMin = np.nanmin(y)
    rectHeight = yMax - yMin
    yRectCoor = yMin
    allRects = []   # this is what will be returned

    # creating rectangle patches
    while index < len(y):

        # if nan exists, then need to create a rectangle patch
        if np.isnan(y[index]):
            xRectCoorIndex = index - 1

            # condition for if first y value is nan
            if index == 0:
                xRectCoorIndex += 1
            
            # condition for if last y value is nan, assumes y is not len 2
            elif index + 1 == len(y):
                xRectCoor = mdates.date2num(dates[xRectCoorIndex])
                coords = (xRectCoor, yRectCoor)
                width = mdates.date2num(dates[xRectCoorIndex + 1]) - mdates.date2num(dates[xRectCoorIndex])
                allRects.append(mpatches.Rectangle(coords, width, rectHeight, color=shortRectColor, alpha=opacityCoeff))
                break
                
            # all other cases
            xRectCoor = mdates.date2num(dates[xRectCoorIndex])

            # checking finding how long the rectangle needs to be--how many consecutive null values
            index += 1
            while np.isnan(y[index]):
                index += 1
            rightEdgeIndex = mdates.date2num(dates[index])

            # making rectangle
            coords = (xRectCoor, yRectCoor)
            width = rightEdgeIndex - xRectCoor
            color = shortRectColor
            if index - xRectCoorIndex > 5:
                color = longRectColor
            allRects.append(mpatches.Rectangle(coords, width, rectHeight, color=color, alpha=opacityCoeff))

        else:
            index += 1

    return allRects

def visualizeMissingValues(dates, arr, fig, ax, wantToMakeNullRects = True):
    '''This function plots an array of values with datetime x axis values onto
    a given axis, showing patches of null values if present.

    Input:
        dates: a numpy array of datetime objs that are the x-axis for the array with missing data to plot
        arr: a numpy array that has missing data
        fig: a matplotlib figure that contains the axis with the plot
        ax: a matplotlib axis that will be plotted upon

    Returns:
        fig: edited matplotlib figure
        ax: edited matplotlib axis
    '''
    ax.plot(dates, arr)

    if wantToMakeNullRects:
        rects = makeNullRects(dates, arr)
        for rect in rects:
            ax.add_patch(rect)

    formatter = mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(), formats=["%Y", "%Y-%b", "%b-%d", "%d %H:%M", "%d %H:%M", "%H:%M"])
    locator = mdates.AutoDateLocator()
    ax.xaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_locator(locator)

    fig.autofmt_xdate()
    return fig, ax

def plotImputedData(dates, nullArr, imputedArr, ax):
    '''This graph plots imputed data as a green dashed line on a given
    matplotlib axis.

    Input:
        dates: a numpy array of datetime objs that are the x-axis for the array with missing data to plot
        nullArr: a numpy array that has missing data
        imputedArr: a numpy array that has some of the missing values imputed
        ax: a matplotlib axis that will be plotted upon
    
    Returns:
        ax: edited matplotlib axis
    '''
    index = 0
    while index < len(nullArr):                                 # looping through arr since it has the null values
        if np.isnan(nullArr[index]):
            # getting the width of the null area
            lenForward = 0
            while np.isnan(nullArr[index + lenForward]):
                lenForward += 1

            # domain to plot is [index-1, index+lenforward]
            domain = list(range(index-1, index+lenForward+1))
            datesToPlot = [dates[i] for i in domain]
            pointsToPlot = [imputedArr[i] for i in domain]
            ax.plot(datesToPlot, pointsToPlot, "g--")       # green dashed line

            # moving index forward past null gap
            index += lenForward
        else:
            index += 1
    return ax


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
%matplotlib qt


feature = "SWTP Total Influent Flow"
# feature = "SWTP Plant 2 Influent Flow"
# feature = "SW_Peak_Flow"
# feature = "SWTP Plant 1 Gravity Flow"
# feature = "Total 168 Hour Rainfall Aggregate"
# feature = "Ozark Aquifer Depth to Water Level (ft)"
# feature = "Springfield Plateau Aquifer Depth to Water Level (ft)"
# feature = "James Gauge Height (ft)"
# feature = "Wilsons Gauge Height (ft)"
# feature = "Fire 168 Hour Rainfall Aggregate"
# feature = "Fire Rainfall (in)"
# feature = "HourlyPressureChange"


df = pd.read_csv("Joined Influent and Rainfall and Weather and Groundwater and Creek Gauge.csv", parse_dates=["DateTime"])
# df = pd.read_csv("Small Gap Imputed Data.csv", parse_dates=["DateTime"])
# df = pd.read_csv("Small Gap Imputed Data Editted.csv", parse_dates=["DateTime"])
# df["SWTP Total Influent Flow"] = np.array([np.nan if x < 3.7 else x for x in df["SWTP Total Influent Flow"]])


# imputedDf = pd.read_csv("Small Gap Imputed Data.csv")
# imputedDf = pd.read_csv("Small Gap Imputed Data Editted.csv")
# imputedDf = pd.read_csv("test.csv")
imputedDf = pd.read_csv("Imputed Data.csv")
# imputedDf = pd.read_csv("New Imputed Data.csv")


dates = np.array(df["DateTime"])
imputedArr = np.array(imputedDf[feature])
nullArr = deepcopy(np.array(df[feature]))

fig, ax = plt.subplots()
fig, ax = visualizeMissingValues(dates, nullArr, fig, ax)
ax = plotImputedData(dates, nullArr, imputedArr, ax)
# ax = plotValidatedValues(ax)
# ax.scatter(testDf["DateTime"], testDf[testDf.columns[-1]], s=8, color="red", marker="x")
ax.set_ylabel(feature, fontsize=16)
ax.set_title("Missing Data in the " + feature, fontsize=20)
# ax.set_xlim([18789.35639453603, 18838.599498146727])
# ax.set_ylim([3.6382304053667287, 10.719097953664527])
plt.show()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
%matplotlib qt

def sign_ae(x, y):
    sign_x = tf.math.sign(x)
    sign_y = tf.math.sign(y)
    delta = x - y
    return sign_x * sign_y * tf.math.abs(delta)
    
def linex_loss(delta, a=3., b=3.):
    if a != 0 and b > 0:
        # loss = 1/(a*a) * (tf.exp(a * delta) - a * delta - 1)
        loss = b * (tf.exp(a * delta) - a * delta - 1)
        return loss
    else:
        raise ValueError("linex_loss error with a or b")
          
def linex_loss_val(y_true, y_pred):
    delta = sign_ae(y_true, y_pred)
    res = linex_loss(delta)
    return tf.math.reduce_mean(res)


def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def sliding_window(X, y, n_test, slide):
    split_point = X.shape[0] - n_test + slide
    train_X , train_y = X[:split_point, :] , y[:split_point, :]
    test_X , test_y = X[split_point:, :] , y[split_point:, :]
    return train_X, train_y, test_X, test_y

def invNormalize(arr, minimum, maximum):
    return (maximum - minimum) * arr + minimum

def predict(model, test_X, test_y, fullData = False):
    # to be able to inverse scale predictions
    df = pd.read_csv("Train and Test Data.csv", usecols=["SWTP Total Influent Flow"])
    if fullData:
        df = pd.read_csv("Imputed Data.csv", usecols=["SWTP Total Influent Flow"])
    arr = np.array(df["SWTP Total Influent Flow"])
    maximum = np.max(arr)
    minimum = np.min(arr)

    #predictions and rescaling to [min, max]
    y_pred = model.predict(test_X)
    y_pred_inv = np.array([invNormalize(x, minimum, maximum) for x in y_pred])
    test_y_inv = np.array([invNormalize(x, minimum, maximum) for x in test_y])
    print("y_pred_inv:",y_pred_inv.shape)
    print("test_y_inv:",y_pred_inv.shape)
    
    return y_pred_inv, test_y_inv

def nnse(predictions, targets):
    term1 = np.sum((predictions - targets) ** 2)
    term2 = np.sum((predictions - np.average(targets)) ** 2)
    nse = 1 - (term1 / term2)
    return 1 / (2 - nse)

def nseAbs(predictions, targets):
    term1 = np.sum(np.abs(predictions - targets))
    term2 = np.sum(np.abs((predictions - np.average(targets))))
    nse = 1 - (term1 / term2)
    return nse

def nse(predictions, targets):
    term1 = np.sum((predictions - targets)**2)
    term2 = np.sum((predictions - np.average(targets))**2)
    nse = 1 - (term1 / term2)
    return nse

features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 
            "Ozark Aquifer Depth to Water Level (ft)", "James Gauge Height (ft)", 'HourlyStationPressure']
dataset = pd.read_csv("Imputed Data.csv", usecols=features)
arr = np.array(dataset["SWTP Total Influent Flow"])
dataset["Target"] = arr         # adding another influent flow feature so that past values can be used to predict future values
values = dataset.values
scaler = MinMaxScaler()         # linear transformation of each feature from [min, max] to [0, 1]
scaled = scaler.fit_transform(values)

n_steps_in = 36
n_steps_out = 24
# n_steps_out = 5
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
train_X, train_y, test_X, test_y = sliding_window(X, y, 8760, 0)    # a full year of data


modelPath = "loss\\linex loss 1.1 best\\model"
# modelPath = "loss\\linex loss 5 hours\\model"
model = keras.models.load_model(modelPath, custom_objects={'linex_loss_val': linex_loss_val})

# modelPath = "loss\\mse loss\\model"
# model = keras.models.load_model(modelPath)

y_pred_inv, y_test_inv = predict(model, test_X, test_y, True)
x = list(range(len(y_pred_inv)))
y = [np.abs(y_pred_inv[i] - y_test_inv[i]) / y_test_inv[i] for i in range(len(y_pred_inv))]
y = [np.average(yi) for yi in y]
print("The average relative error on each forecast is:", np.average(y))
# y = [mean_squared_error(y_pred_inv[i], y_test_inv[i]) for i in range(len(y_pred_inv))]


# plotting metric of future predictions
df = pd.read_csv("Imputed Data.csv", usecols=["DateTime"], parse_dates=["DateTime"])
dates = np.array(df["DateTime"])
dates = dates[-1*test_y.shape[0]-n_steps_out:-1*n_steps_out]

fig, ax = plt.subplots()
formatter = mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(), formats=["%Y", "%Y-%b", "%b-%d", "%d %H:%M", "%d %H:%M", "%H:%M"])
locator = mdates.AutoDateLocator()
ax.xaxis.set_major_formatter(formatter)
ax.xaxis.set_major_locator(locator)
fig.autofmt_xdate()

# ax.set_title("RMSE of 24 Hour Forecasts per Hour", fontsize=20)
# ax.set_ylabel("RMSE", fontsize=16)
fig.suptitle("Absolute Relative Error of {n} Hour Forecasts".format(n=n_steps_out), fontsize=20)
ax.set_title("Average Absolute Relative Error = {avg}".format(avg = round(np.average(y), 4)), fontsize=20)
ax.set_ylabel("Relative Error", fontsize=16)
# ax.plot(dates, np.sqrt(y))  # root mean square error
ax.plot(dates, y) 
# ax.set_xlim([18975.997366488264, 19011.15631337849])
# ax.set_ylim([0, 0.45])
plt.show()

y_pred_inv: (8760, 24)
test_y_inv: (8760, 24)
The average relative error on each forecast is: 0.11388417376972226


In [3]:
features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 
            "Ozark Aquifer Depth to Water Level (ft)", "James Gauge Height (ft)", 'HourlyStationPressure']
dataset = pd.read_csv("Imputed Data.csv", usecols=features)
arr = np.array(dataset["SWTP Total Influent Flow"])
dataset["Target"] = arr         # adding another influent flow feature so that past values can be used to predict future values
values = dataset.values
scaler = MinMaxScaler()         # linear transformation of each feature from [min, max] to [0, 1]
scaled = scaler.fit_transform(values)

n_steps_in = 36
# n_steps_out = 24
# n_steps_out = 5
n_steps_out = 1
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
train_X, train_y, test_X, test_y = sliding_window(X, y, 8760, 0)    # a full year of data


# modelPath = "loss\\linex loss 1.1 best\\model"
# modelPath = "loss\\linex loss 5 hours\\model"
modelPath = "loss\\linex loss 1 hour\\model"
model = keras.models.load_model(modelPath, custom_objects={'linex_loss_val': linex_loss_val})

y_pred_inv, y_test_inv = predict(model, test_X, test_y, True)
x = list(range(len(y_pred_inv)))
y = [np.abs(y_pred_inv[i] - y_test_inv[i]) / y_test_inv[i] for i in range(len(y_pred_inv))]
y = [np.average(yi) for yi in y]
print("The average relative error on each forecast is:", np.average(y))
# y = [mean_squared_error(y_pred_inv[i], y_test_inv[i]) for i in range(len(y_pred_inv))]

print("The nse for inv y is:", nse(y_pred_inv.flatten(), y_test_inv.flatten()))
print("The nnse for inv y is:", nnse(y_pred_inv.flatten(), y_test_inv.flatten()))
print("The nse1 for inv y is:", nseAbs(y_pred_inv.flatten(), y_test_inv.flatten()))
print("The r2 for inv y is:", r2_score(y_pred_inv.flatten(), y_test_inv.flatten()))

y_pred_inv: (8760, 1)
test_y_inv: (8760, 1)
The average relative error on each forecast is: 0.0034403925321362964
The nse for inv y is: 0.9999228377545633
The nnse for inv y is: 0.999922843708116
The nse1 for inv y is: 0.9917735314212088
The r2 for inv y is: 0.9999228371667168


In [15]:
# plotting metric of future predictions
df = pd.read_csv("Imputed Data.csv", usecols=["DateTime", "SWTP Total Influent Flow"], parse_dates=["DateTime"])
dates = np.array(df["DateTime"])
target = np.array(df["SWTP Total Influent Flow"])
dates = dates[-1*test_y.shape[0]-n_steps_out:-1*n_steps_out]
target = target[-1*test_y.shape[0]-n_steps_out:-1*n_steps_out]

fig, ax = plt.subplots()
ax.plot(dates, target, label="True")
for i in range(len(y_pred_inv)):
    correspondingDates = dates[i:i+24]

print(len(y_pred_inv))
print(len(dates))

8760
8760


In [35]:
# change date to cheek forecasted predictions vs actual values on that date
year = "2021"
month = "11"
day = "14"
hour = "23"
dateDesired = year + "-" + month + "-" + day + "T" + hour + ":00:00.000000000"  # string format of how timestamps are stored

# getting index in dates array corresponding to the string dateDesired
index = -1
for i in range(len(dates)):
    if dateDesired == str(dates[i]):
        index = i
        break
if index == -1:
    print("Could not find date!")

rmseValue = np.sqrt(mean_squared_error(y_test_inv[index], y_pred_inv[index]))
relError = np.average(np.abs(y_test_inv[index] - y_pred_inv[index]) / y_test_inv[index])

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(list(range(1, n_steps_out+1)), y_test_inv[index], label="Actual")
ax.plot(list(range(1, n_steps_out+1)), y_pred_inv[index], label="Prediction", color="red")
ax.legend()

ax.set_xlabel("Hours in Advance", fontsize=14)
ax.set_ylabel("Total Influent Flow Rate", fontsize=14)
fig.suptitle("Results for " + dateDesired[:10] + " at " + dateDesired[11:16], fontsize=18)
# ax.set_title("RMSE=" + str(round(rmseValue, 4)), fontsize=18)
ax.set_title("Relative Error=" + str(round(relError, 4)), fontsize=18)
plt.show()

In [ ]:
hourlyRMSE = np.sqrt(y)
# plt.hist(hourlyRMSE, bins=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 40, 50, 60], density=True)
fig, ax = plt.subplots()
ax.hist(hourlyRMSE, bins = [0, 1, 2, 3, 5, 8, 10, 15, 20, 30, 40, 50, 60],density=False)

for rect in ax.patches:
    height = rect.get_height()
    ax.annotate(f'{int(height)}', xy=(rect.get_x()+rect.get_width()/2, height), 
                xytext=(0, 5), textcoords='offset points', ha='center', va='bottom')
ax.set_xticks([0, 1, 2, 3, 5, 8, 10, 15, 20, 30, 40, 50, 60])
ax.set_ylabel("Frequency", fontsize=16)
ax.set_xlabel("RMSE Bins", fontsize=16)
plt.show()